# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fef48318910>
├── 'a' --> tensor([[ 0.7298, -0.9396, -1.1550],
│                   [ 0.2021,  0.0035,  1.3206]])
└── 'x' --> <FastTreeValue 0x7fef48318790>
    └── 'c' --> tensor([[ 0.0562,  0.7622, -0.2006,  0.2263],
                        [ 0.3258,  0.7490,  0.5535,  0.0437],
                        [-0.3504,  0.6729,  0.3956,  0.4682]])

In [4]:
t.a

tensor([[ 0.7298, -0.9396, -1.1550],
        [ 0.2021,  0.0035,  1.3206]])

In [5]:
%timeit t.a

75.5 ns ± 0.288 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fef48318910>
├── 'a' --> tensor([[ 0.3552, -1.3697, -0.3256],
│                   [-0.1514,  0.8657, -1.3179]])
└── 'x' --> <FastTreeValue 0x7fef48318790>
    └── 'c' --> tensor([[ 0.0562,  0.7622, -0.2006,  0.2263],
                        [ 0.3258,  0.7490,  0.5535,  0.0437],
                        [-0.3504,  0.6729,  0.3956,  0.4682]])

In [7]:
%timeit t.a = new_value

83.5 ns ± 2.06 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.7298, -0.9396, -1.1550],
               [ 0.2021,  0.0035,  1.3206]]),
    x: Batch(
           c: tensor([[ 0.0562,  0.7622, -0.2006,  0.2263],
                      [ 0.3258,  0.7490,  0.5535,  0.0437],
                      [-0.3504,  0.6729,  0.3956,  0.4682]]),
       ),
)

In [10]:
b.a

tensor([[ 0.7298, -0.9396, -1.1550],
        [ 0.2021,  0.0035,  1.3206]])

In [11]:
%timeit b.a

70.9 ns ± 0.261 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.7496,  0.0753, -0.8961],
               [-0.9093, -0.0374, -0.9454]]),
    x: Batch(
           c: tensor([[ 0.0562,  0.7622, -0.2006,  0.2263],
                      [ 0.3258,  0.7490,  0.5535,  0.0437],
                      [-0.3504,  0.6729,  0.3956,  0.4682]]),
       ),
)

In [13]:
%timeit b.a = new_value

596 ns ± 3.19 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

961 ns ± 3.99 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.1 µs ± 19.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

169 µs ± 459 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

168 µs ± 2.1 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fee93683f40>
├── 'a' --> tensor([[[ 0.7298, -0.9396, -1.1550],
│                    [ 0.2021,  0.0035,  1.3206]],
│           
│                   [[ 0.7298, -0.9396, -1.1550],
│                    [ 0.2021,  0.0035,  1.3206]],
│           
│                   [[ 0.7298, -0.9396, -1.1550],
│                    [ 0.2021,  0.0035,  1.3206]],
│           
│                   [[ 0.7298, -0.9396, -1.1550],
│                    [ 0.2021,  0.0035,  1.3206]],
│           
│                   [[ 0.7298, -0.9396, -1.1550],
│                    [ 0.2021,  0.0035,  1.3206]],
│           
│                   [[ 0.7298, -0.9396, -1.1550],
│                    [ 0.2021,  0.0035,  1.3206]],
│           
│                   [[ 0.7298, -0.9396, -1.1550],
│                    [ 0.2021,  0.0035,  1.3206]],
│           
│                   [[ 0.7298, -0.9396, -1.1550],
│                    [ 0.2021,  0.0035,  1.3206]]])
└── 'x' --> <FastTreeValue 0x7fee936835e0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37.4 µs ± 150 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fee93683760>
├── 'a' --> tensor([[ 0.7298, -0.9396, -1.1550],
│                   [ 0.2021,  0.0035,  1.3206],
│                   [ 0.7298, -0.9396, -1.1550],
│                   [ 0.2021,  0.0035,  1.3206],
│                   [ 0.7298, -0.9396, -1.1550],
│                   [ 0.2021,  0.0035,  1.3206],
│                   [ 0.7298, -0.9396, -1.1550],
│                   [ 0.2021,  0.0035,  1.3206],
│                   [ 0.7298, -0.9396, -1.1550],
│                   [ 0.2021,  0.0035,  1.3206],
│                   [ 0.7298, -0.9396, -1.1550],
│                   [ 0.2021,  0.0035,  1.3206],
│                   [ 0.7298, -0.9396, -1.1550],
│                   [ 0.2021,  0.0035,  1.3206],
│                   [ 0.7298, -0.9396, -1.1550],
│                   [ 0.2021,  0.0035,  1.3206]])
└── 'x' --> <FastTreeValue 0x7fee936839a0>
    └── 'c' --> tensor([[ 0.0562,  0.7622, -0.2006,  0.2263],
                        [ 0.3258,  0.7490,  0.5535,  0.0437],
                 

In [23]:
%timeit t_cat(trees)

35.1 µs ± 90.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

64.5 µs ± 327 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.0562,  0.7622, -0.2006,  0.2263],
                       [ 0.3258,  0.7490,  0.5535,  0.0437],
                       [-0.3504,  0.6729,  0.3956,  0.4682]],
              
                      [[ 0.0562,  0.7622, -0.2006,  0.2263],
                       [ 0.3258,  0.7490,  0.5535,  0.0437],
                       [-0.3504,  0.6729,  0.3956,  0.4682]],
              
                      [[ 0.0562,  0.7622, -0.2006,  0.2263],
                       [ 0.3258,  0.7490,  0.5535,  0.0437],
                       [-0.3504,  0.6729,  0.3956,  0.4682]],
              
                      [[ 0.0562,  0.7622, -0.2006,  0.2263],
                       [ 0.3258,  0.7490,  0.5535,  0.0437],
                       [-0.3504,  0.6729,  0.3956,  0.4682]],
              
                      [[ 0.0562,  0.7622, -0.2006,  0.2263],
                       [ 0.3258,  0.7490,  0.5535,  0.0437],
                       [-0.3504,  0.6729,  0.3956,  0.4682]],

In [26]:
%timeit Batch.stack(batches)

90.2 µs ± 287 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.0562,  0.7622, -0.2006,  0.2263],
                      [ 0.3258,  0.7490,  0.5535,  0.0437],
                      [-0.3504,  0.6729,  0.3956,  0.4682],
                      [ 0.0562,  0.7622, -0.2006,  0.2263],
                      [ 0.3258,  0.7490,  0.5535,  0.0437],
                      [-0.3504,  0.6729,  0.3956,  0.4682],
                      [ 0.0562,  0.7622, -0.2006,  0.2263],
                      [ 0.3258,  0.7490,  0.5535,  0.0437],
                      [-0.3504,  0.6729,  0.3956,  0.4682],
                      [ 0.0562,  0.7622, -0.2006,  0.2263],
                      [ 0.3258,  0.7490,  0.5535,  0.0437],
                      [-0.3504,  0.6729,  0.3956,  0.4682],
                      [ 0.0562,  0.7622, -0.2006,  0.2263],
                      [ 0.3258,  0.7490,  0.5535,  0.0437],
                      [-0.3504,  0.6729,  0.3956,  0.4682],
                      [ 0.0562,  0.7622, -0.2006,  0.2263],
                   

In [28]:
%timeit Batch.cat(batches)

167 µs ± 726 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

385 µs ± 4.82 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
